In [1]:
import os
import sys
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pixel as pix
import utilities as util
import tile
import importlib
import resnet as rs
import keras
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="2";

In [3]:
landsat, sentinel, dem, label = util.load_data()
tile_size = 10
num_classes = util.get_class_count()
pixel_gen = pix.pixel_gen(landsat, sentinel, dem, label, tile_size, num_classes)
tile_gen = tile.tile_gen(landsat, sentinel, dem, label, tile_size, num_classes)

In [4]:
pixels = util.read_txt(["train_px.txt", "val_px.txt", "test_px.txt"])
train_px, val_px, test_px = (pixels[0], pixels[1], pixels[2])

In [5]:
epochs = 100
batch_size = 25
input_shape = tile_gen.get_tile_shape(reshape=True)
print(input_shape)

(10, 10, 10)


In [8]:
model = rs.ResnetBuilder.build_resnet_18(input_shape,num_classes)
#model = make_model(input_shape)

In [9]:
model.summary()

In [10]:
opt = Adam()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit_generator(generator=tile_gen.tile_generator(train_px, batch_size), 
                    steps_per_epoch=len(train_px) // batch_size, epochs=epochs, verbose=1,
                    validation_data=tile_gen.tile_generator(val_px, batch_size),
                    validation_steps=len(val_px) // batch_size)

Epoch 1/100
20791/20791 [==============================] - 2290s 110ms/step - loss: 1.4693 - acc: 0.5303 - val_loss: 1.4814 - val_acc: 0.5252
Epoch 2/100
20791/20791 [==============================] - 2284s 110ms/step - loss: 1.4688 - acc: 0.5305 - val_loss: 1.4999 - val_acc: 0.5147
Epoch 3/100
20791/20791 [==============================] - 2286s 110ms/step - loss: 1.4682 - acc: 0.5312 - val_loss: 1.4829 - val_acc: 0.5227
Epoch 4/100
20791/20791 [==============================] - 2286s 110ms/step - loss: 1.4676 - acc: 0.5306 - val_loss: 1.5134 - val_acc: 0.5084
Epoch 5/100
20791/20791 [==============================] - 2282s 110ms/step - loss: 1.4673 - acc: 0.5311 - val_loss: 1.4985 - val_acc: 0.5177
Epoch 6/100
20791/20791 [==============================] - 2290s 110ms/step - loss: 1.4663 - acc: 0.5316 - val_loss: 1.4743 - val_acc: 0.5246
Epoch 7/100
20791/20791 [==============================] - 2276s 109ms/step - loss: 1.4660 - acc: 0.5316 - val_loss: 1.5042 - val_acc: 0.5172
Epoch 

20791/20791 [==============================] - 2262s 109ms/step - loss: 1.4563 - acc: 0.5343 - val_loss: 1.5754 - val_acc: 0.5038
Epoch 59/100
20791/20791 [==============================] - 2263s 109ms/step - loss: 1.4570 - acc: 0.5343 - val_loss: 1.5536 - val_acc: 0.5070
Epoch 60/100
20791/20791 [==============================] - 2263s 109ms/step - loss: 1.4560 - acc: 0.5348 - val_loss: 1.5114 - val_acc: 0.5190
Epoch 61/100
20791/20791 [==============================] - 2265s 109ms/step - loss: 1.4561 - acc: 0.5345 - val_loss: 1.4694 - val_acc: 0.5305
Epoch 62/100
20791/20791 [==============================] - 2263s 109ms/step - loss: 1.4567 - acc: 0.5336 - val_loss: 1.4757 - val_acc: 0.5288
Epoch 63/100
20791/20791 [==============================] - 2265s 109ms/step - loss: 1.4552 - acc: 0.5340 - val_loss: 1.5184 - val_acc: 0.5175
Epoch 64/100
20791/20791 [==============================] - 2264s 109ms/step - loss: 1.4556 - acc: 0.5344 - val_loss: 1.5262 - val_acc: 0.5130
Epoch 65/100

In [14]:
model.save('small_cnn_fixed.h5')

# Test Pixels

In [ ]:
tile_gen.evaluate_cnn(model, test_px)

# Test Pixels on specific tiles

In [ ]:
pixel_buckets = { 0: list(), 1: list(), 2: list(), 3: list(), 4: list()}
for pix in test_px:
    pixel_buckets[pix[1]].append(pix)

# Tile 0

In [ ]:
tile_gen.evaluate_cnn(model, pixel_buckets[0])

# Tile 1

In [ ]:
tile_gen.evaluate_cnn(model, pixel_buckets[1])

# Tile 2

In [ ]:
tile_gen.evaluate_cnn(model, pixel_buckets[2])

# Tile 3

In [ ]:
tile_gen.evaluate_cnn(model, pixel_buckets[3])

# Tile 4

In [ ]:
tile_gen.evaluate_cnn(model, pixel_buckets[4])